In [ ]:
# TP - Classification d'Expressions Faciales avec CNN

# Étape 1 : Vérification GPU et Imports
import os
import pathlib
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.model_selection
import sklearn.metrics
import sklearn.utils
import tensorflow as tf
from tensorflow import keras

# Vérification GPU
print("GPU disponible :", tf.test.is_gpu_available())

# Étape 2 : Chargement des données
def load_data(csv_path, images_dir):
    """
    Charger les images et les labels à partir d'un fichier CSV

    Args:
    - csv_path: Chemin vers le fichier CSV des labels
    - images_dir: Répertoire contenant les images

    Returns:
    - images: Tableau numpy des images
    - labels: Tableau numpy des labels
    """
    # Lecture du CSV
    df = pd.read_csv(csv_path)

    # Préparation des listes pour stocker images et labels
    images = []
    labels = []

    # Parcours du dataframe
    for index, row in df.iterrows():
        # Chemin complet de l'image
        img_path = os.path.join(images_dir, row['filename'])

        # Charger l'image en niveaux de gris et redimensionner
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (48, 48))  # Taille standard pour les CNN
            images.append(img)
            labels.append(row['emotion'])

    return np.array(images), np.array(labels)

# Charger les données
images_dir = 'facial_expressions/data/images'
csv_path = 'facial_expressions/data/legend.csv'
images, labels = load_data(csv_path, images_dir)

# Étape 3 : Exploration du dataset
# Distribution des classes
unique_labels, label_counts = np.unique(labels, return_counts=True)
plt.figure(figsize=(10, 5))
plt.bar(unique_labels, label_counts)
plt.title('Distribution des Classes')
plt.xlabel('Émotions')
plt.ylabel('Nombre d\'images')
plt.show()

# Affichage de quelques images
plt.figure(figsize=(15, 5))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(images[i], cmap='gray')
    plt.title(f'Émotion: {labels[i]}')
    plt.axis('off')
plt.tight_layout()
plt.show()

# Prétraitement des données
# Normalisation et reshape
images = images.reshape(-1, 48, 48, 1) / 255.0

# Encodage one-hot des labels
labels = keras.utils.to_categorical(labels)

# Séparation train/test
train_images, test_images, train_labels, test_labels = (
    sklearn.model_selection.train_test_split(
        images, labels, test_size=0.2, stratify=np.argmax(labels, axis=1)
    )
)

# Construction du modèle CNN
def create_cnn_model(input_shape, num_classes):
    model = keras.Sequential([
        # Première couche de convolution
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Dropout(0.25),

        # Deuxième couche de convolution
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Dropout(0.25),

        # Troisième couche de convolution
        keras.layers.Conv2D(128, (3, 3), activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Dropout(0.25),

        # Couche de flattening
        keras.layers.Flatten(),

        # Couches fully connected
        keras.layers.Dense(512, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.5),

        # Couche de sortie
        keras.layers.Dense(num_classes, activation='softmax')
    ])

    return model

# Création du modèle
model = create_cnn_model((48, 48, 1), num_classes=labels.shape[1])

# Compilation du modèle
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Calcul des poids de classe
class_weights = sklearn.utils.class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(np.argmax(labels, axis=1)),
    y=np.argmax(labels, axis=1)
)

# Clip des poids de classe
class_weights = np.clip(class_weights, 0, 5)
class_weights_dict = dict(enumerate(class_weights))

# Entraînement du modèle
history = model.fit(
    train_images, train_labels,
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    class_weight=class_weights_dict,
    verbose=1
)

# Évaluation du modèle
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

# Prédictions
y_pred = model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(test_labels, axis=1)

# Matrice de confusion
cm = sklearn.metrics.confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matrice de Confusion')
plt.xlabel('Prédictions')
plt.ylabel('Vraies Classes')
plt.show()

# Courbes d'apprentissage
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.legend()
plt.tight_layout()
plt.show()